# Chapter 7. Information Processing Capacity

In this chapter, we will learn how to calculate the information processing capacity (IPC), an extended metric of the memory capacity introduced in the previous chapter.
As in the previous chapter, we will also explore the relationship between the properties of dynamical systems, such as rank and dynamics, and the resulting IPC.

**Note:** In the latter part of this chapter, an environment with GPU support is recommended.
If your local PC does not have a GPU, it is recommended to run it on Google Colaboratory.

## Introduction

IPC is an extended metric of memory capacity proposed by J. Dambre et al.<sup>[1]</sup>, which evaluates what kind of computations (memory and nonlinearity) a dynamical system performs on input time series.
As in the previous chapter, we consider a single-input $N$-dimensional dynamical system $x[k]$ represented by the following equations and the output $\hat{y}[k]$ obtained through a certain linear mapping $g: \mathbb{R}^N \to \mathbb{R}$:

$$
\renewcommand{\Tau}{\mathrm{T}}
\renewcommand{\Zeta}{\mathrm{Z}}
\begin{align*}
x[k+1] &= f \left(x[k],\zeta[k+1]\right) \\
\hat{y}[k] &= g(x[k])
,\end{align*}
$$

where the input time series $\zeta[k]$ is assumed to be zero-mean, stationary, and i.i.d.
Here, we introduce a new capacity $\mathrm{C}$ defined by the following equation:

$$
\begin{align*}
\mathrm{C}[x, z] := \mathrm{R}^2[z, x]
.\end{align*}
$$

As shown in the equation, $\mathrm{C}(x, z)$ is a metric that quantifies how well the target time series $z$ can be reconstructed from $x$, calculated using the coefficient of determination $\mathrm{R}^2$, and it takes values in the range [0, 1].
The memory function, learned in the previous chapter, is expressed using $\mathrm{C}$ as follows:

$$
\begin{align*}
\mathrm{MF}[\tau] &= \mathrm{C}[x, \zeta^\tau]
.\end{align*}
$$

The memory capacity $\mathrm{MC}$ is calculated by summing $\mathrm{C}$ over all past inputs by varying $\tau$.
In other words, memory capacity can be interpreted as a metric that quantifies how well the **linear** transformation of past inputs $[\zeta[k], \zeta[k-1], \zeta[k-2],~\ldots]$ can be recovered from the current state $x[k]$ (or its linear transformation).
On the other hand, the calculation of IPC evaluates how well reconstruction can be achieved, extending the scope to include **nonlinear** transformations.

Now, how should we construct the nonlinear transformation $z$ of past inputs that we aim to evaluate? The points to consider here are the orthogonality and completeness of the target time series, meaning that all patterns must be considered without duplication.
This is because allowing linearly dependent targets would make the total capacity arbitrarily large.
In the calculation of memory capacity, the simple summation of the values of each memory function $\mathrm{C}[x, \zeta^\tau]$ was valid because the i.i.d. nature of the input was assumed.
That is, for $\tau_1 \neq \tau_2$, $\zeta^{\tau_1}$ and $\zeta^{\tau_2}$ were linearly independent and orthogonal ($\mathrm{E}[\zeta^{\tau_1} \zeta^{\tau_2}] = 0$).

In the calculation of IPC, a tool called **orthogonal polynomials**<sup>[2]</sup> is used to construct target time series comprehensively without duplication.
Orthogonal polynomials have an integer parameter called **degree**.
Generally, the larger the value of $d$, the stronger the nonlinearity (conversely, $d=0$ corresponds to a constant, and $d=1$ corresponds to a linear transformation).
Orthogonal polynomials of degree $d$ are denoted as $\mathcal{P}_d$.

First, the first-order target time series used in memory capacity can be expressed again in the following form using the first-degree orthogonal polynomial $\mathcal{P}_1$:

$$
\begin{align*}
\mathcal{P}_1(\zeta^0),~\mathcal{P}_1(\zeta^1),~\mathcal{P}_1(\zeta^2),~\mathcal{P}_1(\zeta^3),~\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^5),~\ldots
.\end{align*}
$$

Next, let us consider second-order target time series.
Here, using the second-degree orthogonal polynomial $\mathcal{P}_2$, they are enumerated as follows:

$$
\begin{align*}
&\mathcal{P}_2(\zeta^0),~\mathcal{P}_2(\zeta^1),~\mathcal{P}_2(\zeta^2),~\mathcal{P}_2(\zeta^3),~\mathcal{P}_2(\zeta^4),~\mathcal{P}_2(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^1),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^2),~\mathcal{P}_{1}(\zeta^0)\mathcal{P}_1(\zeta^3),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^2),~\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^3),~\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^3),~\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^4)\mathcal{P}_1(\zeta^5),\ldots
.\end{align*}
$$

Here, note that as second-order target time series, not only the elements using $\mathcal{P}_2$ but also the products of $\mathcal{P}_{1}$ (i.e., $1+1=2$) are included.
Since they are orthogonal, all target time series are linearly independent, and the inner product between them is $0$.

Third-order target time series are also enumerated similarly.
Since there are three patterns of addition that result in a total degree of 3: $3,~2+1, 1+1+1$, the number of considered patterns increases as follows:

$$
\begin{align*}
&\mathcal{P}_3(\zeta^0),~\mathcal{P}_3(\zeta^1),~\mathcal{P}_3(\zeta^2),~\mathcal{P}_3(\zeta^3),~\mathcal{P}_3(\zeta^4),~\mathcal{P}_3(\zeta^5),~\ldots\\
&\mathcal{P}_2(\zeta^0)\mathcal{P}_1(\zeta^1),~\mathcal{P}_2(\zeta^0)\mathcal{P}_1(\zeta^2),~\mathcal{P}_2(\zeta^0)\mathcal{P}_1(\zeta^3),~\mathcal{P}_2(\zeta^0)\mathcal{P}_1(\zeta^4),~\mathcal{P}_2(\zeta^0)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_2(\zeta^1)\mathcal{P}_1(\zeta^2),~\mathcal{P}_2(\zeta^1)\mathcal{P}_1(\zeta^3),~\mathcal{P}_2(\zeta^1)\mathcal{P}_1(\zeta^4),~\mathcal{P}_2(\zeta^1)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_2(\zeta^2)\mathcal{P}_1(\zeta^3),~\mathcal{P}_2(\zeta^2)\mathcal{P}_1(\zeta^4),~\mathcal{P}_2(\zeta^2)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_2(\zeta^3)\mathcal{P}_1(\zeta^4),~\mathcal{P}_2(\zeta^3)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_2(\zeta^4)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^2),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^3),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^3),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^0)\mathcal{P}_1(\zeta^4)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^3),~\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^1)\mathcal{P}_1(\zeta^4)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^4),~\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^2)\mathcal{P}_1(\zeta^4)\mathcal{P}_1(\zeta^5),~\ldots\\
&\mathcal{P}_1(\zeta^3)\mathcal{P}_1(\zeta^4)\mathcal{P}_1(\zeta^5),~\ldots
.\end{align*}
$$

The combinations continue to increase explosively for $d = 4$ and beyond as well.
We generalize these and define a target time series $\zeta^{D,\Tau}$ using the following equation, with $D=\{d_1,d_2,~\ldots,~d_m\}$ being a set of integers representing degrees and $\Tau=\{\tau_1, \tau_2,~\ldots,~\tau_m\}$ being a set of integers representing time delays, both having the same number of elements:

$$
\begin{align*}
\zeta^{D,\Tau}[k] :=& \mathcal{P}_{d_1}(\zeta[k-\tau_1])\mathcal{P}_{d_2}(\zeta[k-\tau_2])\cdots \mathcal{P}_{d_m}(\zeta[k-\tau_m]) \\
=& \prod_{i=1}^m \mathcal{P}_{d_i}(\zeta[k-\tau_i])
.\end{align*}
$$

The $d$-th order IPC $\mathrm{C}^d$ is defined as the sum of capacities for target time series corresponding to $D$ such that $\sum_i d_i = d$:

$$
\begin{align*}
\mathrm{C}^d[x, \zeta] := \sum_{\substack{D~\mathrm{s.t.}\\ \sum_i d_i=d}} \sum_{\Tau} \mathrm{C}[x, \zeta^{D,\Tau}]
.\end{align*}
$$

From the definition, we have $\mathrm{MC}=\mathrm{C}^1$.
This is why IPC is an extended metric of memory capacity.
The total capacity $\mathrm{C}^\mathrm{tot}$ is defined as:

$$
\begin{align*}
\mathrm{C}^\mathrm{tot}[x, \zeta] &:= \sum_{d=1}^{\infty} \mathrm{C}^d[x, \zeta]
.\end{align*}
$$

J. Dambre et al.<sup>[1]</sup> demonstrated the following inequality regarding the upper limit of this IPC (derivation is an advanced exercise):

$$
\begin{align*}
\mathrm{C}^\mathrm{tot}[x, \zeta] \leq r \leq N
,\end{align*}
$$

where $r$ represents the rank of the dynamical system.
This indicates that, similar to memory capacity, the upper limit of IPC is constrained by the number of linearly independent dimensions of the internal states.

The results derived from this metric provide particularly important insights for physical reservoir computing (PRC).
In PRC, the number of sensors typically installed directly corresponds to the dimensionality of the internal states, and the number of linearly independent sensor time series directly represents the upper limit of IPC.
Moreover, since IPC comprehensively evaluates what kinds of transformations are performed, it is also useful for assessing the characteristics of the physical system itself.

## Exercises and demonstrations

Let's move on to the exercises and demonstrations.
Along with the basic libraries from the previous chapter, you can import and use the `ESN`, `Linear`, and `narma_func` we implemented earlier.
Please run the following cell.

You can view the implementations of `ESN`, `Linear`, and `narma_func` by uncommenting the lines after `import inspect` or by using `...?? / ??...`.

In [ ]:
import itertools
import math
import sys

import numpy as np
import scipy as sp

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/rc-bootcamp/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/rc-bootcamp/
    else:
        pass
        %cd /content/
        !git clone --branch en https://github.com/rc-bootcamp/rc-bootcamp.git
        %cd /content/rc-bootcamp/
else:
    sys.path.append(".")

from ipc_module.helper import visualize_dataframe
from ipc_module.profiler import UnivariateProfiler, UnivariateViewer
from utils.reservoir import ESN, Linear
from utils.style_config import Figure, plt
from utils.tester import load_from_chapter_name
from utils.tqdm import tqdm, trange

test_func, show_solution = load_from_chapter_name("07_information_processing_capacity")

# Uncomment it to see the implementations of `Linear` and `ESN`.
# import inspect
# print(inspect.getsource(Linear))
# print(inspect.getsource(ESN))

# Or just use ??.../...?? (uncomment the following lines).
# Linear??
# ESN??

### 1. Legendre polynomials and verification of orthogonality

So far, we have explained orthogonal polynomials denoted as $\mathcal{P}$, but let's introduce specific polynomials for discussion.
These orthogonal polynomials depend on the distribution of the input time series.
For example, if the input time series $\zeta[k]$ follows a uniform random distribution $\mathcal{U}([-1, 1])$, we can use the [Legendre polynomials](https://en.wikipedia.org/wiki/Legendre_polynomials) defined by the following recurrence relation:

$$
\begin{align*}
(n+1)\mathcal{P}_{n+1}(z) &= (2n+1)z\mathcal{P}_n(z) - n\mathcal{P}_{n-1}(z)
,\end{align*}
$$

where $\mathcal{P}_0(z)=1$ and $\mathcal{P}_1(z)=z$.
Expanding this equation yields the following polynomials:

$$
\begin{align*}
\mathcal{P}_0(z) &= 1 \\
\mathcal{P}_1(z) &= z \\
\mathcal{P}_2(z) &= \frac{1}{2}(3z^2-1) \\
\mathcal{P}_3(z) &= \frac{1}{2}(5z^3-3z) \\
\mathcal{P}_4(z) &= \frac{1}{8}(35z^4-30z^2+3) \\
\mathcal{P}_5(z) &= \frac{1}{8}(63z^5-70z^3+15z) \\
\mathcal{P}_6(z) &= \frac{1}{48}(231z^6-315z^4+105z^2-5)
.\end{align*}
$$

Orthogonality, on the other hand, can be evaluated based on the result of inner product calculations.
Now, consider two target time series $\zeta^A := \zeta^{D_A,\Tau_A}$ and $\zeta^B := \zeta^{D_B,\Tau_B}$, from which two target time series matrices $\Zeta^A = [\zeta^A[0]; \zeta^A[1];~\ldots;~\zeta^A[T-1]] \in \mathbb{R}^{T \times 1}$ and $\Zeta^B = [\zeta^B[0]; \zeta^B[1];~\ldots;~\zeta^B[T-1]] \in \mathbb{R}^{T \times 1}$ are extracted for $T$ steps.
The inner product $I(\Zeta^A, \Zeta^B)$ of $\Zeta^A$ and $\Zeta^B$ can be calculated as follows:

$$
\begin{align*}
I(\Zeta^A, \Zeta^B) &= \sum_{t=1}^{T} \frac{\Zeta^A_t}{\sqrt{\sum_{t=1}^T (\Zeta^A_t)^2}} \cdot \frac{\Zeta^B_t}{\sqrt{\sum_{t=1}^T (\Zeta^B_t)^2}}
.\end{align*}
$$

The expected value of $I$ based on orthogonality is calculated as follows:

$$
\begin{align*}
\mathrm{E}[I(\Zeta^A, \Zeta^B)] &= \begin{cases}
1 & \mathrm{if}~\zeta^A = \zeta^B~(\Leftrightarrow  D_A=D_B \land \Tau_A=\Tau_B) \\
0 & \mathrm{if}~\zeta^A \perp \zeta^B~(\mathrm{otherwise}) \\
\end{cases}
.\end{align*}
$$

Let us implement the Legendre polynomials and inner product calculations in the exercises to verify these properties.

Q1.1

Based on the above recurrence relation, complete the method `Legendre._calc` in the class `Legendre` to compute the Legendre polynomial. Note that `Legendre._calc` calculates $\mathcal{P}_n(\Zeta)$ for the time series $\Zeta$. The result will be validated against a time series of length $T$ sampled from a uniform random distribution $\mathcal{U}([-1, 1])$.

- `Legendre._calc`
  - Argument(s):
    - `n`: `int`
      - `n >= 0`
  - Operation(s):
    - Update `self._cache[n]`
- $10 \leq T \leq 10^{3}$, $1\leq n \leq 20$

<details><summary>tips</summary>

$$
\begin{align*}
n\mathcal{P}_{n}(z) &= (2n-1)z\mathcal{P}_{n-1}(z) - (n-1)\mathcal{P}_{n-2}(z) ~\mathrm{for}~n \geq 2 \\
.\end{align*}
$$
</details>

In [ ]:
class Legendre(object):
    def __init__(self, xs):
        self.xs = xs
        self._caches = {}
        self._caches[0] = 1
        self._caches[1] = self.xs

    def __getitem__(self, deg):
        assert deg >= 0
        if deg not in self._caches:
            self._caches[deg] = self._calc(deg)
        return self._caches[deg]

    def _calc(self, n: int):
        # TODO Use `self.xs` and `self[n-1]`, `self[n-2]` to calculate the n-th Legendre polynomial.
        ...


def solution(us, n):
    # DO NOT CHANGE HERE.
    poly = Legendre(us)
    return poly[n]


test_func(solution, "01_01")
# show_solution("01_01", "Legendre")  # Uncomment it to see the solution.

Q1.2.

Based on the above equation, complete the method `calc_inner_product` to calculate the inner product of two time series $A \in \mathbb{R}^{T}$ and $B \in \mathbb{R}^{T}$.

- `calc_inner_product`
  - Argument(s):
    - `a`: `np.ndarray`
      - `shape`: `(t,)`
      - `dtype`: `np.float64`
  - Return(s):
    - `b`: `np.ndarray`
      - `shape`: `(t,)`
      - `dtype`: `np.float64`
- $10 \leq T \leq 10^3$

In [ ]:
def calc_inner_product(a, b):
    # TODO Calculate and return the inner product between vectors a and b.
    ...


test_func(calc_inner_product, "01_02")
# show_solution("01_02")  # Uncomment it to see the solution.

Now, let's verify their orthogonality.
First, let's check that we can reproduce the [figure of Legendre polynomials from Wikipedia](https://en.wikipedia.org/wiki/Legendre_polynomials#/media/File:Legendrepolynomials6.svg).

In [ ]:
# https://en.wikipedia.org/wiki/Legendre_polynomials
us = np.linspace(-1, 1, 1000)
poly = Legendre(us)
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
for deg in range(1, 6):
    ax.plot(us, poly[deg], label=r"$\mathcal{P}_" + f"{{{deg}}}$", color=f"C{deg}")
ax.legend(
    loc="upper left",
    fontsize=12,
    bbox_to_anchor=(1.025, 1.0),
    borderaxespad=0,
    frameon=False,
)
ax.tick_params(axis="both", labelsize=12)

None

Next, generate a time series using the uniform random distribution $\mathcal{U}([-1, 1])$, create various target time series using time delay and Legendre polynomials, and calculate the inner products between them.
Each element of `degree_delay_list` specifies a pair of degree sequence $D$ and time delay sequence $\Tau$ in $\zeta^{D,\Tau}$.
Try changing them in various ways and check if orthogonality is maintained.

In [ ]:
seed = 1234
t_washout, t_sample = 100, 10000

rnd = np.random.default_rng(seed)
t_total = t_washout + t_sample
us = rnd.uniform(-1, 1, t_total)
poly = Legendre(us)

degree_delay_list = [
    ([1], [0]),
    ([1], [10]),
    ([1, 1], [1, 2]),
    ([2], [0]),
    ([3], [5]),
]  # You can add or change more combinations if you want.


def create_poly(args):
    degrees, taus = args
    out = 1
    for deg, tau in zip(degrees, taus, strict=True):
        out *= poly[deg][t_washout - tau : t_total - tau]
    return out


def create_label(args):
    degrees, taus = args
    out = r"$"
    for d, t in zip(degrees, taus, strict=True):
        if t == 0:
            out += f"P_{{{d}}}(\\zeta)"
        else:
            out += f"P_{{{d}}}(\\zeta^{{{t}}})"
    out += r"$"
    return out


length = len(degree_delay_list)
polys = list(map(create_poly, degree_delay_list))
labels = list(map(create_label, degree_delay_list))
products = np.zeros((length, length))

for idx, idy in itertools.product(range(length), range(length)):
    products[idx, idy] = calc_inner_product(polys[idx], polys[idy])

fig = Figure(figsize=(8, 6))
ax = fig[0]
im, cb = ax.plot_matrix(
    products,
    cmap="Blues",
    vmin=0,
    vmax=1,
    aspect="equal",
    colorbar=True,
)
ax.set_xticks(range(length))
ax.set_yticks(range(length))
ax.set_xticklabels(labels, fontsize=10)
ax.set_yticklabels(labels, fontsize=10)
cb.ax.tick_params(labelsize=12)

None

Q1.3 (Advanced)

- In the above demo, the value of the inner product $I$ changes depending on the time series length `t_sample`. Confirm that when the time series length is short, the value of $I$ does not approach 0 even if they are orthogonal.
- Instead of using the uniform random distribution $\mathcal{U}([-1, 1])$, when using the standard normal distribution $\mathcal{N}(0, 1)$, [Hermite polynomials](https://en.wikipedia.org/wiki/Hermite_polynomials#Recurrence_relation) must be used instead<sup>[2]</sup>. Implement the class `Hermite` to calculate Hermite polynomials and similarly verify their orthogonality.

### 2. Implementation and verification

Now, let's actually calculate the IPC.
First, prepare an ESN with $N=10$ and the activation function $\tanh$, along with an input time series $\zeta[k]$ that follows the uniform random distribution $\mathcal{U}([-1, 1])$, and sample its dynamics $x[k]$.
To ensure asymmetry, $\zeta[k]$ is scaled to take values in the range $[0, 1]$.
(The input is made asymmetric because $\tanh$ is an odd function, and for symmetric input, only the odd-order components of the capacity appear [verification is an advanced task]).

In [ ]:
seed = 5678
dim = 10
t_washout = 1000
t_sample = 1000000
t_total = t_washout + t_sample
display = True

rnd = np.random.default_rng(seed)
w_in = Linear(1, dim, bound=0.1, bias=0.0, rnd=rnd)
net = ESN(dim, sr=0.1, f=np.tanh, p=1, rnd=rnd)

x0 = np.zeros((dim,))
us = rnd.uniform(-1, 1, (t_total, 1))

x = x0
xs = np.zeros((t_total, *x0.shape))
for idx in trange(t_total, display=display):
    x = net(x, w_in(0.5 * us[idx] + 0.5))
    # x = net(x, w_in(us[idx]))  # Uncomment it for the symmetric case.
    xs[idx] = x

print("us:", us.shape)
print("xs:", xs.shape)

As with the memory function calculations covered in the previous chapter, IPC is also computed using SVD.
Thus, the same code can be reused (if you haven't reviewed it, please refer back to the previous chapter).
The following `calc_regression_and_rank` and `calc_capacity` functions are reused implementations from the previous chapter.

In [ ]:
def calc_regression_and_rank(X):
    T, N = X.shape[-2:]
    X = X - X.mean(axis=-2, keepdims=True)
    U, sigma, _V = np.linalg.svd(X, full_matrices=False)
    eps = np.finfo(X.dtype).eps
    sigma_sq_max = np.max(sigma * sigma, axis=-1, keepdims=True)
    eps = sigma_sq_max * (eps * max(T, N))
    mask = sigma > eps
    rank = mask.sum(axis=-1)
    return U, mask, rank


def calc_capacity(U, mask, zeta):
    uzeta = U.swapaxes(-2, -1) @ zeta
    dot = ((uzeta * uzeta) * mask[..., None]).sum(axis=-2)
    var = (zeta * zeta).sum(axis=-2)
    r2 = dot / var
    return r2

First, let's perform SVD and check the rank $r$.
The rank serves as the upper limit of $\mathrm{C}^\mathrm{tot}$.

In [ ]:
U, mask, rank = calc_regression_and_rank(xs[t_washout:])
print("rank:", rank)

poly = Legendre(us)

#### Calculation of first-order capacity

First, let's calculate the first-order capacity $C^1$.
Specifically, since $\mathcal{P}_1(z) = z$, the first-order target time series can be calculated as follows:

$$
\begin{align*}
\zeta^{0}, \zeta^{1}, \zeta^{2}, \zeta^{3},~\ldots
.\end{align*}
$$

In [ ]:
delay1_max = 10

taus = np.arange(0, delay1_max + 1)
c1 = np.zeros(len(taus))
for idx, tau in enumerate(tqdm(taus)):
    zeta = poly[1][t_washout - tau : t_total - tau]
    c1[idx] = calc_capacity(U, mask, zeta)[..., 0]

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.plot(taus, c1, label=r"$\zeta^1$", color="C0", marker="o")
ax.set_xlim(-0.5, delay1_max + 0.5)
ax.set_xlabel(r"$\tau$", fontsize=14)
ax.set_ylabel(r"$\mathrm{C}[x,\zeta^\tau]$", fontsize=14)
ax.tick_params(axis="both", which="major", labelsize=12)
ax.set_title(r"$\mathrm{C}^1$=" + f"{c1.sum():.3f}", fontsize=14)

None

#### Calculation of second-order capacity

In the case of the second order, there are two possible combinations: $D=\{1,1\}$ and $D=\{2\}$.
Using $\tau_1$ and $\tau_2$ such that $\tau_1 \leq \tau_2$, we can comprehensively prepare the orthogonal polynomials $z^{\tau_1, \tau_2}$ in the following form:

$$
\begin{align*}
z^{\tau_1, \tau_2} = \begin{cases}
\mathcal{P}_2(\zeta^{\tau_1}) &= \frac{3}{2}(\zeta^{\tau_1})^2 - \frac{1}{2} & \mathrm{if}~\tau_1 = \tau_2 \\
\mathcal{P}_1(\zeta^{\tau_1})\mathcal{P}_1(\zeta^{\tau_2}) &= \zeta^{\tau_1} \zeta^{\tau_2} & \mathrm{if}~\tau_1 \leq \tau_2 \\
\end{cases}
.\end{align*}
$$

In [ ]:
delay2_max = 8

taus = np.arange(0, delay2_max + 1)
c2 = np.zeros((len(taus), len(taus)))

cands = list(itertools.product(enumerate(taus), repeat=2))
for (idx, tau1), (idy, tau2) in tqdm(cands):
    if not (idx <= idy):
        continue
    if idx == idy:
        zeta = poly[2][t_washout - tau1 : t_total - tau1]
    else:
        zeta1 = poly[1][t_washout - tau1 : t_total - tau1]
        zeta2 = poly[1][t_washout - tau2 : t_total - tau2]
        zeta = zeta1 * zeta2
    c2[idx, idy] = calc_capacity(U, mask, zeta)[..., 0]

fig = Figure(figsize=(8, 6))
ax = fig[0]
im, cb = ax.plot_matrix(
    c2,
    x=taus,
    y=taus,
    cmap="viridis",
    zscale="log",
    vmax=1,
    vmin=1e-3,
    aspect="equal",
    colorbar=True,
    xticks_kws=dict(num_tick=len(taus)),
    yticks_kws=dict(num_tick=len(taus)),
)
ax.grid(False)
ax.set_xlim(-1, len(taus))
ax.set_ylim(-1, len(taus))
ax.set_xlabel(r"$\tau_2$", fontsize=14)
ax.set_ylabel(r"$\tau_1$", fontsize=14)
ax.tick_params(axis="both", which="major", labelsize=12)
cb.ax.tick_params(labelsize=12)
cb.set_label(r"$\mathrm{C}[x,z^{\tau_1,\tau_2}]$", fontsize=14)
ax.set_title(r"$\mathrm{C}^2$=" + f"{c2.sum():.3f}", fontsize=14)

None

#### Calculation of third-order capacity

For the third order, there are three possible combinations: $D=\{1,1,1\}$, $D=\{2, 1\}$, and $D=\{3\}$.
Similar to the second-order case, we prepare $\tau_1, \tau_2, \tau_3$ such that $\tau_1 \leq \tau_2 \leq \tau_3$ and construct the third-order target time series $z^{\tau_1, \tau_2, \tau_3}$ as follows:

$$
\begin{align*}
z^{\tau_1, \tau_2, \tau_3} = \begin{cases}
\mathcal{P}_3(\zeta^{\tau_1}) &= \frac{5}{2}(\zeta^{\tau_1})^3 - \frac{3}{2}\zeta^{\tau_1} & \mathrm{if}~\tau_1 = \tau_2 = \tau_3 \\
\mathcal{P}_2(\zeta^{\tau_1})\mathcal{P}_1(\zeta^{\tau_3}) &= \left(\frac{3}{2}(\zeta^{\tau_1})^2 - \frac{1}{2}\right)\zeta^{\tau_3} & \mathrm{if}~\tau_1 = \tau_2 < \tau_3 \\
\mathcal{P}_1(\zeta^{\tau_1})\mathcal{P}_2(\zeta^{\tau_2}) &= \zeta^{\tau_1}\left(\frac{3}{2}(\zeta^{\tau_2})^2 - \frac{1}{2}\right) & \mathrm{if}~\tau_1 < \tau_2 = \tau_3 \\
\mathcal{P}_1(\zeta^{\tau_1})\mathcal{P}_1(\zeta^{\tau_2})\mathcal{P}_1(\zeta^{\tau_3}) &= \zeta^{\tau_1}\zeta^{\tau_2}\zeta^{\tau_3} & \mathrm{if}~\tau_1 < \tau_2 < \tau_3 \\
\end{cases}
.\end{align*}
$$

In [ ]:
delay3_max = 6

taus = np.arange(0, delay3_max + 1)
c3 = np.zeros((len(taus), len(taus), len(taus)))

cands = list(itertools.product(enumerate(taus), repeat=3))
for (idx, tau1), (idy, tau2), (idz, tau3) in tqdm(cands):
    if not (idx <= idy <= idz):
        continue
    if idx == idy == idz:
        zeta = poly[3][t_washout - tau1 : t_total - tau1]
    elif idx == idy:
        zeta1 = poly[2][t_washout - tau1 : t_total - tau1]
        zeta2 = poly[1][t_washout - tau3 : t_total - tau3]
        zeta = zeta1 * zeta2
    elif idy == idz:
        zeta1 = poly[1][t_washout - tau1 : t_total - tau1]
        zeta2 = poly[2][t_washout - tau2 : t_total - tau2]
        zeta = zeta1 * zeta2
    else:
        zeta1 = poly[1][t_washout - tau1 : t_total - tau1]
        zeta2 = poly[1][t_washout - tau2 : t_total - tau2]
        zeta3 = poly[1][t_washout - tau3 : t_total - tau3]
        zeta = zeta1 * zeta2 * zeta3
    c3[idx, idy, idz] = calc_capacity(U, mask, zeta)[..., 0]


num_col = math.ceil(len(taus) / 2)
grid_size = (2, num_col)
fig = Figure(figsize=(grid_size[1] * 3, grid_size[0] * 3))
fig.create_grid(*grid_size, hspace=0.35, wspace=0.3)

for pos in range(len(taus)):
    ax = fig[pos // num_col, pos % num_col]
    res = ax.plot_matrix(
        c3[pos],
        x=taus,
        y=taus,
        cmap="viridis",
        zscale="log",
        vmax=1,
        vmin=1e-3,
        aspect="equal",
        colorbar=len(taus) == (pos + 1),
        xticks_kws=dict(num_tick=len(taus)),
        yticks_kws=dict(num_tick=len(taus)),
    )
    ax.grid(False)
    ax.set_xlim(-1, len(taus))
    ax.set_ylim(-1, len(taus))
    if pos % num_col == 0:
        ax.set_ylabel(r"$\tau_2$", fontsize=14)
    if (pos // num_col) == grid_size[0] - 1:
        ax.set_xlabel(r"$\tau_3$", fontsize=14)
    ax.tick_params(axis="both", which="major", labelsize=12)
    ax.set_title(r"$\tau_1$=" + f"{taus[pos]}", fontsize=14)
    if len(taus) == (pos + 1):
        cb = res[1]
        cb.ax.set_position([0.9, 0.1, 0.03, 0.8])
        cb.ax.tick_params(labelsize=12)
        cb.set_label(r"$\mathrm{C}[x,z^{\tau_1,\tau_2,\tau_3}]$", fontsize=14)
if len(taus) < (grid_size[0] * grid_size[1]):
    for pos in range(len(taus), grid_size[0] * grid_size[1]):
        fig.delaxes(fig[pos // num_col, pos % num_col])
fig.suptitle(r"$\mathrm{C}^3$=" + f"{c3.sum():.3f}", fontsize=16)

None

Finally, we sum up the calculated $\mathrm{C}^1, \mathrm{C}^2, \mathrm{C}^3$ to compute the overall IPC $\mathrm{C}^\mathrm{tot}$.

In [ ]:
c_tot = np.sum(c1) + np.sum(c2) + np.sum(c3)
print("total_capacity", c_tot, "rank", rank)

Q2.1. (Advanced)

- Read reference [1] and verify the derivation of $\mathrm{C}^\mathrm{tot} \leq r$.
- Verify that the even-order components of $\mathrm{C}^d$ (particularly $d=2$) vanish when the input is made symmetric. Also, consider and explain the reason for this.
- Change the activation function to an even function and similarly verify its behavior.

Q2.2. (Advanced)

- Implement code to efficiently output a set of degrees $D=\{d_i\}_i$ such that $\sum_{i} d_i = d$ for a given integer $d$ (Cf. [Young tableau](https://en.wikipedia.org/wiki/Young_tableau)).
    - Alternatively, you may use the [`make_degree_list`](https://github.com/rc-bootcamp/ipc-module/blob/main/src/ipc_module/helper.py#L60) implemented in the `ipc-module` mentioned later.
    - It can be imported and used with `from ipc_module.helper import make_degree_list`.
- Implement code to enumerate all possible combinations of time delays $\Tau=\{\tau_i\}_i$ within the range of $\tau_\mathrm{max} \geq 1$, given a set of degrees $D$ and a maximum time delay $\tau_\mathrm{max}$.

### 3. Fast IPC computation using libraries

#### Environmental setup

As confirmed so far, calculating the IPC requires an exhaustive search over combinations of orthogonal polynomials and time delays, leading to an explosive increase in computational cost as the degree grows.
Additionally, as the degree increases, the number of ways to partition the degree grows exponentially, making it very challenging to implement each case individually as done in the previous section (it is known that the general term for the number of partitions asymptotically approaches $p(n)\sim\frac{1}{4\sqrt{3}n}e^{\pi\sqrt{\frac{2n}{3}}}$<sup>[3]</sup>).
Therefore, in this exercise, we will learn how to calculate IPC using the library [`ipc-module`](https://rc-bootcamp.github.io/ipc-module/), which was developed within the laboratory.

`ipc-module` supports GPU-accelerated tensor computation libraries such as `pytorch` and `cupy` in addition to `numpy`, enabling more efficient calculations compared to CPU-only operations.
It also provides functions for organizing and visualizing results, allowing you to easily examine the IPC of a given dynamical system.
It is [published on PyPI](https://pypi.org/project/ipc-module/) and can be installed via `pip install ipc-module`, but in this notebook, we directly include the source code to make it easier to check and modify locally.
For specific implementations, refer to the Python code in the `./ipc_module` folder.

Note that the following code will run on a CPU as is, but it will take considerable time.
Therefore, if you have access to a GPU environment, **using a GPU is strongly recommended** (if you don't have one locally, we recommend running it on Google Colaboratory).
In that case, follow the guide below for additional setup.

<details><summary>Preparations for calculations in a GPU environment</summary>

`pytorch` is very convenient, so the following explains how to set it up.

1. In an online environment (Google Colaboratory):

   Even with the free version, GPU can be used by default, and `pytorch` is already installed, so no additional setup is required. However, you can verify that the GPU is enabled using the following steps:
   "Edit" > "Notebook settings" > "Hardware accelerator" > "GPU"

2. In a local environment (for uv):

   For NVIDIA GPUs, first install the drivers.
   You can check if they are installed using the `nvidia-smi` command.
   If not installed, you can download them from the official [distribution page](https://www.nvidia.com/en-us/drivers/).
   Then, you can install them using the following command:
   ```bash
   uv sync --extra gpu
   ```
   This will automatically start the installation of `pytorch`.
    ```
</details>

#### Explanation of basic operations

Once ready, let's perform calculations using `ipc_module`.
First, prepare an ESN with $N=50$ dimensions, vary the spectral radius from 0.1 to 1.7 in increments of 0.1, and simultaneously sample its dynamics.
As before, prepare an input time series $\zeta$ from a uniform random distribution $\mathcal{U}([-1, 1])$, scale it to the range $[0, 1]$ to make it asymmetric, and input it into the ESN.
(The memory requirement is large, so if you encounter a memory shortage error, reduce `t_sample` or `dim` as needed.
In general, longer sample lengths lead to higher calculation accuracy.)

In [ ]:
seed = 5678
dim = 50
t_washout = 10000
t_sample = 100000
srs = np.linspace(0.1, 1.7, 17)
t_total = t_washout + t_sample
display = True

rnd = np.random.default_rng(seed)
w_in = Linear(1, dim, bound=0.1, bias=0.0, rnd=rnd)
net = ESN(dim, sr=srs[:, None], f=np.tanh, p=1, rnd=rnd)

x0 = np.zeros((srs.shape[0], dim))
us = rnd.uniform(-1, 1, (t_total, 1))

x = x0
xs = np.zeros((t_total, *x0.shape))
for idx in trange(t_total, display=display):
    x = net(x, w_in(0.5 * us[idx] + 0.5))
    xs[idx] = x

print("us:", us.shape)
print("xs:", xs.shape)

The [`UnivariateProfiler`](https://rc-bootcamp.github.io/ipc-module/profiler/#ipc_module.profiler.UnivariateProfiler) is a class equipped with various methods for calculating IPC.
You pass a one-dimensional input time series and the corresponding state time series, and then calculate the IPC within the specified range of degrees and time delays using [`UnivariateProfiler.calc`](https://rc-bootcamp.github.io/ipc-module/profiler/#ipc_module.profiler.UnivariateProfiler.calc).

<details><summary> Details of the arguments</summary>

- `us`: `np.ndarray | torch.Tensor | cupy.ndarray`
    - Input time series
    - Must have the shape `(t, ..., 1)`
- `xs`: `np.ndarray | torch.Tensor | cupy.ndarray`
    - Corresponding state time series
    - Must have the shape `(t, ..., N)`
- `poly_name`: `str`
    - Name of the polynomial to use
        - [`Legendre`](https://rc-bootcamp.github.io/ipc-module/polynomial/#ipc_module.polynomial.Legendre): Legendre polynomial
        - [`Hermite`](https://rc-bootcamp.github.io/ipc-module/polynomial/#ipc_module.polynomial.Hermite): Hermite polynomial
        - [`GramSchmidt`](https://rc-bootcamp.github.io/ipc-module/polynomial/#ipc_module.polynomial.GramSchmidt): Polynomial expansion using the Gram-Schmidt method
- `offset`: `int`
    - Time delay offset (specifies the index where $t=0$)
    - Default is 0
- `surrogate_num`: `int`
    - Number of surrogate samples
    - Default is 1000
- `surrogate_seed`: `int`
    - Seed for surrogate samples
    - Default is 0
- `axis1`: `int`
    - Axis corresponding to the time dimension
    - The same axis is used for both `us` and `xs`
    - Default is 0
- `axis2`: `int`
    - Axis corresponding to the state dimension
    - The same axis is used for both `us` and `xs`
    - Default is -1
</details>

Let’s try using it.
First, create an instance of the `UnivariateProfiler` class, named `profiler`.
If you are in an environment where a GPU cannot be used, set `use_gpu` to `False`.

In [ ]:
use_gpu = True  # NOTE: Set it to False to run on CPU.

if use_gpu:
    import torch

    assert torch.cuda.is_available(), "CUDA is not available"
    us_c = torch.from_numpy(us).cuda()
    xs_c = torch.from_numpy(xs).cuda()
    args = (us_c, xs_c)
else:
    args = (us, xs)

profiler = UnivariateProfiler(
    *args,
    "Legendre",
    offset=t_washout,
    surrogate_num=1000,
    axis1=0,
    axis2=-1,
)

Here is a simple explanation of the algorithm:
1. When creating an instance of the `UnivariateProfiler` class, the state time series provided as the second argument (in this case, `xs`) is normalized, followed by performing SVD and simultaneously measuring the rank (the implementation is almost the same as `calc_regression_and_rank`).
2. Immediately afterward, shuffled indices to be used for surrogate data (described later) are generated (as specified by `surrogate_num`).
3. The `UnivariateProfiler.calc` method is executed to calculate the IPC within the specified range of degrees and time delays based on the SVD results. The calculation of the target time series (and the orthogonal polynomials required for its construction) is done lazily, meaning it is computed only when needed, and the results are cached.

Since steps 1 and 2 have already been completed in the previous cell, let’s execute the `UnivariateProfiler.calc` method in the following cell to calculate the first-order capacity $C^1$.

In [ ]:
profiler.calc(1, 1001)

In this cell, $\Tau \in \{\{0\}, \{1\}, \{2\},~\ldots,~\{1000\}\}$, that is, for each target time series $z \in \{\zeta^0, \zeta^1, \zeta^2,~\ldots,~\zeta^{1000}\}$, $\mathrm{C}[x,z]$ is being calculated (`zero_offset=False` is an option specifying 1-based indexing).
The calculation results can be retrieved in the form of `profiler[key]` by specifying the degree $D$ (note that the content of `key` is a `tuple`).

In [ ]:
delays, ipc, surr = profiler[(1,)]

print("delays:", *delays[:3], "...", *delays[-3:])
print("ipc:", ipc.shape)
print("surr:", surr.shape)

In this way, `profiler` holds three pieces of information for each `key`.
The first, `delays`, is a list of $\Tau$.
`ipc` stores the calculated $\mathrm{C}[x,z]$ in the form of a `np.array`, where the SR corresponds to the second axis.
`surr` represents $\mathrm{C}[x,z]$ for the surrogate data calculated simultaneously.
First, let's check the contents of `ipc`.

In [ ]:
time_step = 301

fig = Figure(figsize=(8, 6))
ax = fig[0]
im, cb = ax.plot_matrix(
    ipc[:time_step, :, 0],
    y=np.array(delays)[:time_step, 0],
    x=srs,
    aspect="auto",
    cmap="jet",
    vmin=1e-4,
    vmax=1,
    zscale="log",
    yticks_kws=dict(num_tick=4),
    xticks_kws=dict(num_tick=3),
)
ax.set_xlabel("SR", fontsize=14)
ax.set_ylabel(r"$\tau$", fontsize=14)
ax.tick_params(axis="both", which="major", labelsize=12)
cb.ax.tick_params(labelsize=12)
cb.set_label(r"$\mathrm{C}[x,\zeta^\tau]$", fontsize=14)
ax.set_title(r"$D=\{1\}$", fontsize=14)

None

This is none other than the memory function learned in the previous chapter.
This graph displays the color range on a logarithmic scale with a minimum value of $10^{-4}$.
Even when $\tau$ is sufficiently large, the values do not become completely zero but remain slightly small.
This phenomenon is called **spurious correlation**, which often arises due to the constraints of numerical computations that can only handle finite data.
The values of capacity $\mathrm{C}$ in regions suspected to be spurious correlation are very small, but they can have a non-negligible impact in calculating IPC, since a large number of $\mathrm{C}$ types must be summed (e.g., $\mathrm{C}^\mathrm{tot}>r$ may occur).

**Surrogate data** is used to distinguish between "significant" components and "insignificant" spurious correlations.
Surrogate data is generated by shuffling the original time series, preserving the statistical properties while eliminating temporal dependencies.
Here, surrogate data is prepared in the number specified by `surrogate_num`, and the capacity $\mathrm{C}$ is measured in the same way, with the maximum value used as the threshold.
Only components exceeding this threshold are considered distinguishable from random noise.
This method is known as the random-shuffle method<sup>[4]</sup>, originally introduced in reference [5].
For 1000 data points, this can be regarded as a test with a significance level of $1/1000=0.001$.
Let's examine the surrogate data for the component with SR=1.0 as an example.

In [ ]:
sr_id = 9  # 9 is the index of the SR = 1.0 in `srs`.
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for value in surr[:, sr_id, :]:
    ax.line_y(value, color="#333333", alpha=0.5, lw=0.1)
ax.line_y(surr[:, sr_id, :].max(), color="red", lw=1)
ax.plot(np.arange(0, 1001), ipc[:, 9, 0], lw=1)
ax.set_yscale("log")
ax.set_ylim([None, 1e-2])  # Comment it out to zoom out.
ax.set_xlim([0, 1000])
ax.set_ylabel(r"$\mathrm{C}[x,\zeta^\tau]$", fontsize=14)
ax.set_xlabel(r"$\tau$", fontsize=14)
ax.tick_params(axis="both", which="major", labelsize=12)
ax.set_title(f"SR={srs[sr_id]:.2f}", fontsize=14)

None

The gray lines represent the capacity $\mathrm{C}$ for each surrogate data, and the red line shows the maximum value among them.
In `ipc_module`, the maximum value of the surrogate data is used as a reference, and a constant multiple of it is set as the threshold to extract significant components (the scaling is often needed because stricter criteria are required when the number of target time series is large).
Let's set the threshold using the surrogate data and redraw the previous graph.
The components below the threshold should appear as hollow.
At the same time, vary the constant multiplier `max_scale` to see how the graph changes with the threshold size.

In [ ]:
time_step = 301
max_scale = 1.0

ipc_trunc = ipc * (ipc > surr.max(axis=0, keepdims=True) * max_scale)
fig = Figure(figsize=(8, 6))
ax = fig[0]
im, cb = ax.plot_matrix(
    ipc_trunc[:time_step, :, 0],
    y=np.array(delays)[:time_step, 0],
    x=srs,
    aspect="auto",
    cmap="jet",
    vmin=1e-4,
    vmax=1,
    zscale="log",
    yticks_kws=dict(num_tick=4),
    xticks_kws=dict(num_tick=3),
)
ax.set_xlabel("SR", fontsize=14)
ax.set_ylabel(r"$\tau$", fontsize=14)
ax.tick_params(axis="both", which="major", labelsize=12)
cb.ax.tick_params(labelsize=12)
cb.set_label(r"$\mathrm{C}[x,\zeta^\tau]$", fontsize=14)
ax.set_title(r"$D=\{1\}$" + f", scale={max_scale:.2f}", fontsize=14)

None

Now that we have calculated and examined the details for $d=1$ as an example, let’s perform similar calculations for other degrees.
The next cell calculates the capacity for $d=2,3,4,5$.
For each, $\tau_\mathrm{max}=300,50,30,15$ is specified.
For example, when $D=\{1,1\}$, the combinations of $\{\tau_1, \tau_2\}$ that are candidates for $\Tau$ are $\{0,1\}, \{0,2\},~\ldots,~\{0,300\}, \{1,2\}, \{1,3\}, \{1,4\},~\ldots,~\{1,300\},~\ldots,~\{299,300\}$.
In this way, all combinations of time delays below $\tau_\mathrm{max}=300$ are exhaustively calculated.
Since the number of combinations is large, this may take some time, so please wait patiently.
Once the calculation is complete, the list of calculated $D$ will be displayed (you can check it with `profiler.keys()`).

In [ ]:
degrees = [2, 3, 4, 5]
taus = [300, 50, 30, 15]
for deg, tau in zip(degrees, taus, strict=True):
    profiler.calc(deg, tau + 1)

print(profiler.keys())

Once the above cell completes, let's save the calculation results.
In general, calculating IPC takes considerable time, so it is advisable to save results periodically to avoid losing them.
Using the [`UnivariateProfiler.save`](https://rc-bootcamp.github.io/ipc-module/profiler/#ipc_module.profiler.UnivariateViewer.save) method, results can be saved to a file in either `npz` or `pkl` format (`npz` is recommended for better compression and easier verification).
The format is determined by the file extension.
You can include additional information to save using `**kwargs`.
Here, let's also save the spectral radii `srs`.

In [ ]:
profiler.save("./result/ipc_asym.npz", srs=srs)

The [`UnivariateViewer`](https://rc-bootcamp.github.io/ipc-module/profiler/#ipc_module.profiler.UnivariateViewer) class is used to retrieve data.
`UnivariateViewer` is the parent class of `UnivariateProfiler` and can retrieve results in the same way as `UnivariateProfiler`.
However, note that it does not retain the original time series or the results of the SVD, so additional calculations cannot be performed (the `calc` function cannot be called).
In the cell below, confirm that the same results can be obtained using the `UnivariateViewer` instance `viewer` as when using `profiler`.

In [ ]:
viewer = UnivariateViewer("./result/ipc_asym.npz")
srs = viewer.info["srs"]  # NOTE: Keyword options are stored on `info`.
print(viewer.keys())
delays, ipc, surr = viewer[(1,)]

print("delays:", *delays[:3], "...", *delays[-3:])
print("ipc:", ipc.shape)
print("surr:", surr.shape)
print("srs:", srs)

#### Data visualization and analysis

Unlike the case of $d=1$, visualization for higher orders is not straightforward.
For $d=2$, as handled in the previous section, $C$ could be visualized as a color map on the $(\tau_1, \tau_2)$ plane.
However, for higher orders, more graphs are required, making it challenging.
To address this, `ipc_module` provides several functions for visualization.
The [`visualize_dataframe`](https://rc-bootcamp.github.io/ipc-module/helper/#ipc_module.helper.visualize_dataframe) function, used in the next cell, is a function that visualizes the total capacity as a bar graph (using degree/component as groups).

<details><summary> Details of the arguments</summary>

- `ax`: `Axes`
    - The Axes to draw on
- `df`: `polars.DataFrame`
    - The DataFrame to visualize
- `ranks`: `Any | None`
    - A list of ranks
    - If provided, the values are filtered accordingly
- `xticks`: `Any | None`
    - Values for the x-axis
- `group_by`: `str`
    - The grouping method for visualization
        - `degree`: Grouped by degree $d$
        - `component`: Grouped by the set of degrees $D$
        - `detail`: Grouped by the set of degrees $D$ and the set of time delays $\Tau$ (Note: visualization may take time if `threshold` is not specified!)
- `threshold`: `float`
    - Threshold for components summarized as `rest`
- `sort_by`: `Any`
    - Sorting method
        - `np.nanmax`: Sort by maximum value
        - `np.nanmean`: Sort by mean value
        - `np.nansum`: Sort by total value
- `cmap`: `str`
    - Color map
</details>

In [ ]:
df, rank = viewer.to_dataframe(max_scale=2.0)  # NOTE: Threshold is scaled by max_scale.
fig, axes = plt.subplots(1, 2, figsize=(16, 6), gridspec_kw=dict(hspace=0.5))

for idx, group_by in enumerate(["degree", "component"]):
    ax = axes[idx]
    visualize_dataframe(
        ax,
        df,
        xticks=srs,
        threshold=0.1,
        cmap="tab10",
        group_by=group_by,  # NOTE: Either "degree" or "component" are available.
        fontsize=12,
    )
    ax.legend(
        loc="upper right",
        fontsize=12,
        bbox_to_anchor=(0.99, 0.9),
        borderaxespad=0,
        frameon=False,
    )
    ax.plot(srs, rank, ls=":", color="k")
    ax.set_xticks([0.0, 0.5, 1.0, 1.5])
    ax.set_xlabel("SR", fontsize=14)
    ax.set_ylabel(r"$\mathrm{C}$", fontsize=14)
axes[0].set_title(r"group by $d$: degree")
axes[1].set_title(r"group by $D$: the set of degree")

None

As mentioned earlier in the above cell, the [`UnivariateViewer.to_dataframe`](https://rc-bootcamp.github.io/ipc-module/profiler/#ipc_module.profiler.UnivariateViewer.to_dataframe) method can be used to retrieve the calculation results in the [`polars.DataFrame`](https://docs.pola.rs/py-polars/html/reference/dataframe/) format.
[`polars`](https://pola.rs/) is a data analysis library similar to [`pandas`](https://pandas.pydata.org/), but it operates much faster.

In [ ]:
df, rank = viewer.to_dataframe(max_scale=2.0)  # NOTE: Threshold is scaled by max_scale.
df

Refer to the [official documentation](https://docs.pola.rs/user-guide/getting-started/) for detailed usage of `polars`.
The next section introduces representative visualization methods.

##### Visualization for $|D|=1$

If the number of elements in $D$ is 1, i.e., $D \in \{\{1\}, \{2\}, \{3\},~\ldots\}$, the capacity $\mathrm{C}$ can be visualized as a one-dimensional graph.
By specifying a negative value as an argument to `viewer.to_dataframe`, you can extract only those with the absolute value of the number of elements in $D$ (e.g., `df = viewer.to_dataframe(-1)`).
In the graph below, $\mathrm{C}[x, \mathcal{P}_d(\zeta^\tau)]$ is visualized for the data of each spectral radius.
Additionally, instead of summing the elements in the `DataFrame`, you can use the `viewer.total` method to calculate the total capacity.

In [ ]:
max_scale = 2.0
degrees = [1, 2, 3]
sr_ids = [4, 9, 14]

df, rank = viewer.to_dataframe(-1, max_scale=0.0)  # NOTE: No truncation.

grid_size = (len(sr_ids), len(degrees))
fig, axes = plt.subplots(
    *grid_size, figsize=(grid_size[1] * 4, grid_size[0] * 3), gridspec_kw=dict(hspace=0.1, wspace=0.1)
)

for (idy, sr_id), (idx, degree) in itertools.product(enumerate(sr_ids), enumerate(degrees)):
    scale = viewer.calc_surr_max((degree,), max_scale=max_scale)[sr_id, 0]
    capacity = viewer.total((degree,), max_scale=max_scale)[sr_id]
    ax = axes[idy, idx]
    df_sub = df.filter(df["degree"] == degree).sort("del_0")  # NOTE: Filter by degree.
    delays = df_sub["del_0"]
    ipc = df_sub[f"ipc_{sr_id}"]
    ax.plot(
        delays,
        ipc,
        color=f"C{idx}",
        lw=1,
        label=f"SR={srs[sr_id]:.2f}",
    )
    ax.line_y(scale, color="red", lw=1, ls="--")
    ax.set_yscale("log")
    ax.set_ylim([1e-4, 1.1])
    ax.tick_params(axis="both", which="major", labelsize=12)
    if idy == 0:
        ax.set_title(f"$d={degree}$", fontsize=14)
    if idx == 0:
        ax.set_ylabel("SR=" + f"{srs[sr_id]:.2f}", fontsize=14)
    else:
        ax.set_yticklabels([])
    if idy < len(sr_ids) - 1:
        ax.set_xticklabels([])
    else:
        ax.set_xlabel(r"$\tau$", fontsize=14)
    ax.text(
        0.95,
        0.95,
        "C={:.2f}".format(capacity),
        fontsize=12,
        ha="right",
        va="top",
        transform=ax.transAxes,
    )
fig.suptitle(r"$\mathrm{C}[x,\mathcal{P}_d(\zeta^\tau)]$" + f", scale={max_scale:.2f}", fontsize=16)
None

##### Visualization for $D=\{1,1\}$

Now, let us consider the case where $D=\{1,1\}$.
Unlike when $|D|=1$, since $D$ has two elements, it can be visualized on a two-dimensional plane.
By using `df = viewer.to_dataframe((1, 1))`, you can obtain a `DataFrame` that extracts only the components for $D=\{1,1\}$.

In [ ]:
max_scale = 2.0

df, rank = viewer.to_dataframe((1, 1), max_scale=max_scale)
capacities = viewer.total((1, 1), max_scale=max_scale)
grid_size = (4, 5)
fig = Figure(figsize=(grid_size[1] * 3, grid_size[0] * 3))
fig.create_grid(*grid_size, hspace=0.3, wspace=0.3)

pos = len(srs)
ax_last = fig[pos // grid_size[1], pos % grid_size[1]]
ax_last.create_grid(1, 2, width_ratios=[1, 20])
cax = ax_last[0]
cax.tick_params(labelsize=12)
for idx, sr in enumerate(srs):
    ax = fig[idx // grid_size[1], idx % grid_size[1]]
    df_pivot = df.sort("del_0", "del_1").pivot(
        "del_1",
        index="del_0",
        values=f"ipc_{idx}",
    )
    mat = df_pivot[:, 1:]
    index = df_pivot[:, 0]
    columns = list(map(int, df_pivot.columns[1:]))
    ax.plot_matrix(
        mat,
        index=index,
        column=columns,
        cmap="viridis",
        zscale="log",
        vmin=1e-4,
        vmax=1,
        aspect="equal",
        xticks_kws=dict(num_tick=7),
        yticks_kws=dict(num_tick=7),
        colorbar=(idx == len(srs) - 1),
        cax=cax if (idx == len(srs) - 1) else None,
    )
    ax.set_xlim(-0.5, 50.5)
    ax.set_ylim(-0.5, 50.5)
    ax.tick_params(axis="both", which="major", labelsize=12)
    ax.text(
        0.95,
        0.95,
        "C={:.2f}".format(capacities[idx]),
        fontsize=12,
        ha="right",
        va="top",
        transform=ax.transAxes,
    )
    ax.set_title(f"SR={sr:.2f}", fontsize=14)

for idx in range(len(srs), grid_size[0] * grid_size[1]):
    fig.delaxes(fig[idx // grid_size[1], idx % grid_size[1]])

#### Application example: confirming the effect of input symmetry

Finally, as an example to demonstrate the effectiveness of IPC, let’s examine the impact on symmetric input.
Using the same ESN conditions as before, provide the input to the ESN while keeping the scale at $[-1, 1]$.

In [ ]:
seed = 5678
dim = 50
t_washout = 10000
t_sample = 100000
srs = np.linspace(0.1, 1.7, 17)
t_total = t_washout + t_sample
display = True

rnd = np.random.default_rng(seed)
w_in = Linear(1, dim, bound=0.1, bias=0.0, rnd=rnd)
net = ESN(dim, sr=srs[:, None], f=np.tanh, p=1, rnd=rnd)

x0 = np.zeros((srs.shape[0], dim))
us = rnd.uniform(-1, 1, (t_total, 1))

x = x0
xs = np.zeros((t_total, *x0.shape))
for idx in trange(t_total, display=display):
    x = net(x, w_in(us[idx]))  # NOTE: Use us[idx] for the symmetric case
    xs[idx] = x

print("us:", us.shape)
print("xs:", xs.shape)

We will measure the IPC under the same conditions as before.
This will also take some time, so please wait.
The results will be saved as `./output/ipc_symm.npz`.

In [ ]:
use_gpu = True  # NOTE: Set it to False to run on CPU.

if use_gpu:
    import torch

    assert torch.cuda.is_available(), "CUDA is not available"
    us_c = torch.from_numpy(us).cuda()
    xs_c = torch.from_numpy(xs).cuda()
    args = (us_c, xs_c)
else:
    args = (us, xs)

profiler = UnivariateProfiler(
    *args,
    "Legendre",
    offset=t_washout,
    surrogate_num=1000,
    axis1=0,
    axis2=-1,
)

degrees = [1, 2, 3, 4, 5]
taus = [1000, 300, 50, 30, 15]
for deg, tau in zip(degrees, taus, strict=True):
    profiler.calc(deg, tau + 1)

print(profiler.keys())

profiler.save("./result/ipc_symm.npz", srs=srs)

The following cell loads data from both `ipc_asym.npz` and `ipc_symm.npz` and compares them in a figure.
For `ipc_symm.npz`, which uses symmetric input, the even-order components are expected to vanish because the activation function $\tanh$ is an odd function.
What will the result be?

In [ ]:
files = ["./result/ipc_asym.npz", "./result/ipc_symm.npz"]

fig, axes = plt.subplots(1, len(files), figsize=(16, 6), gridspec_kw=dict(hspace=0.5))
for idx, file in enumerate(files):
    viewer = UnivariateViewer(file)
    srs = viewer.info["srs"]
    df, rank = viewer.to_dataframe(max_scale=2.0)  # NOTE: Threshold is scaled by max_scale.
    ax = axes[idx]
    visualize_dataframe(
        ax,
        df,
        xticks=srs,
        threshold=0.1,
        cmap="tab10",
        group_by="component",
        fontsize=12,
    )
    ax.legend(
        loc="upper right",
        fontsize=12,
        bbox_to_anchor=(0.99, 0.9),
        borderaxespad=0,
        frameon=False,
    )
    ax.plot(srs, rank, ls=":", color="k")
    ax.set_xticks([0.0, 0.5, 1.0, 1.5])
    ax.set_xlabel("SR", fontsize=14)
    ax.set_ylabel(r"$\mathrm{C}$", fontsize=14)
    ax.set_title(file, fontsize=16)

None

As in the previous chapter, let’s plot and compare the bifurcation diagram along with the calculation of the conditional Lyapunov exponent.

In [ ]:
eps = 1e-4
net.sr = np.linspace(0.1, 1.7, 161)[:, None]

x0 = np.zeros((2, net.sr.shape[0], dim))
us = rnd.uniform(-1, 1, (t_total, 1))

t_washout, t_sample = 1000, 20000
ts = np.arange(-t_washout, t_sample)

x = x0
xs = np.zeros((t_total, *x0.shape[1:]))
lmbds = np.zeros((t_sample, net.sr.shape[0]))
for idx, t in enumerate(tqdm(ts, display=display)):
    if t == 0:
        pert = rnd.uniform(-1, 1, x[0].shape)
        pert = pert / np.linalg.norm(pert, axis=-1, keepdims=True)
        x[1] = x[0] + pert * eps
    x = net(x, w_in(us[idx]))
    xs[idx] = x[0]
    if t >= 0:
        x_org, x_per = x[0], x[1]
        x_diff = x_per - x_org
        d_post = np.linalg.norm(x_diff, axis=-1, keepdims=True)
        lmbd = np.log(np.abs(d_post / eps))
        x_per[:] = x_org + x_diff * (eps / d_post)
        lmbds[idx - t_washout] = lmbd[..., 0]


def get_maxima_and_minima(xs, **kwargs):
    id_maxima = sp.signal.find_peaks(xs, **kwargs)[0]
    id_minima = sp.signal.find_peaks(-xs, **kwargs)[0]
    return id_maxima, id_minima


fig, axes = plt.subplots(2, 1, sharex=True, figsize=(8, 10), gridspec_kw=dict(hspace=0.05))
axl = axes[0]
axl.set_xticklabels([])
for idx, sr in enumerate(net.sr):
    id_maxima, id_minima = get_maxima_and_minima(xs[t_washout:, idx, 0])
    id_all = np.concatenate([id_maxima, id_minima])
    peaks = xs[t_washout:, idx, 0][id_all]
    axl.scatter(sr * np.ones(peaks.shape[0]), peaks, marker=".", s=0.01, color="k")
axl.tick_params(axis="both", which="major", labelsize=12)
axl.set_ylabel(r"$x_0[k]$", fontsize=14)
axl.set_yticks([-1.0, 0.0, 1.0])
axl.set_ylim(-1.1, 1.1)

axr = axes[0].twinx()
axr.plot(net.sr, lmbds.mean(axis=0), "o-", color="red", label="MLE")
axr.set_yticks([-0.2, 0.0, 0.2])
axr.set_ylim(-0.22, 0.22)
axr.set_ylabel(r"MLE: $\lambda$", fontsize=14)
axr.set_xticklabels([])
axr.tick_params(axis="both", which="major", labelsize=12)

viewer = UnivariateViewer("./result/ipc_symm.npz")
srs = viewer.info["srs"]
df, rank = viewer.to_dataframe(max_scale=2.0)  # NOTE: Threshold is scaled by max_scale.
ax = axes[1]
visualize_dataframe(
    ax,
    df,
    xticks=srs,
    threshold=0.1,
    cmap="tab10",
    group_by="component",
    fontsize=12,
)
ax.legend(
    loc="upper right",
    fontsize=12,
    bbox_to_anchor=(0.99, 0.9),
    borderaxespad=0,
    frameon=False,
)
ax.set_xlabel("SR", fontsize=14)
ax.set_ylabel(r"$\mathrm{C}$", fontsize=14)

for ax in [axl, axr, axes[1]]:
    ax.plot(srs, rank, ls=":", color="k")
    ax.set_xticks([0.0, 0.5, 1.0, 1.5, 2.0])
    ax.set_xlim(srs.min() - 0.1, srs.max() + 0.1)
fig.align_labels()

- Use `narma_func` to measure the IPC for NARMA10 and reproduce the results in Figure 3 of [5].
- Compare the IPC of NARMA and ESN, and discuss the conditions under which the ESN can solve NARMA.

## References

[1] Dambre, J., Verstraeten, D., Schrauwen, B., & Massar, S. (2012). *Information Processing Capacity of Dynamical Systems*. Scientific Reports, 2(1), 514. https://doi.org/10.1038/srep00514

[2] Xiu, D., & Karniadakis, G. E. (2002). *The Wiener--Askey Polynomial Chaos for Stochastic Differential Equations*. SIAM Journal on Scientific Computing, 24(2), 619–644. https://doi.org/10.1137/S1064827501387826

[3] Hardy, G. H., & Ramanujan, S. (1918). *Asymptotic Formulaæ in Combinatory Analysis*. Proceedings of the London Mathematical Society, s2-17(1), 75–115. https://doi.org/10.1112/plms/s2-17.1.75

[4] Theiler, J., Eubank, S., Longtin, A., Galdrikian, B., & Doyne Farmer, J. (1992). *Testing for Nonlinearity in Time Series: The Method of Surrogate Data*. Physica D: Nonlinear Phenomena, 58(1), 77–94. https://doi.org/10.1016/0167-2789(92)90102-S

[5] Kubota, T., Takahashi, H., & Nakajima, K. (2021). *Unifying Framework for Information Processing in Stochastically Driven Dynamical Systems*. Physical Review Research, 3(4), 043135. https://doi.org/10.1103/PhysRevResearch.3.043135